In [1]:
import pandas as pd
import os
import plotly.offline as pyoff
import plotly.graph_objs as go

# Investigate data

In [123]:
meta_order = pd.read_csv('data/data_order.csv')
meta_order['CHECKOUT_DATE'] = pd.to_datetime(meta_order['CHECKOUT_DATE'], format='%Y-%m-%d')

# drop irrelevant keys
meta_order = meta_order.drop(['VARIANT_CASE_PRICE_CENTS', 'REGION_ID', 'BRAND_ID', 'ORDER_ID', 'STORE_ID', 'PRODUCT_VARIANT_ID'], axis=1)

meta_order.head(10)

,PRODUCT_ID,QUANTITY,CHECKOUT_DATE
0,4048,1,2022-01-25
1,4661,1,2022-01-24
2,3956,1,2022-01-25
3,2552,4,2022-01-25
4,3668,2,2022-01-25
5,3668,5,2022-01-25
6,3924,1,2022-01-25
7,5071,1,2022-02-24
8,3174,1,2022-01-25
9,3155,2,2022-01-24


## Investigate the sale of a single product

In [128]:
product_ids = meta_order['PRODUCT_ID'].unique()
id = product_ids[3]

In [150]:
# Fill out missing days

product_orders = meta_order.loc[meta_order['PRODUCT_ID'] == id].sort_values(by='CHECKOUT_DATE')
product_orders = product_orders.groupby(['CHECKOUT_DATE', 'PRODUCT_ID']).sum().sort_values(by='CHECKOUT_DATE').reset_index()

r = pd.date_range(start="2022-01-01", end="2022-04-30", freq="D")
fill_values = {"PRODUCT_ID": id, "QUANTITY": 0}
product_orders = product_orders.set_index('CHECKOUT_DATE').reindex(r).fillna(value=fill_values).rename_axis('CHECKOUT_DATE').reset_index()

product_orders

,CHECKOUT_DATE,PRODUCT_ID,QUANTITY
0,2022-01-01,2552.0,0.0
1,2022-01-02,2552.0,3.0
2,2022-01-03,2552.0,20.0
3,2022-01-04,2552.0,35.0
4,2022-01-05,2552.0,331.0
...,...,...,...
115,2022-04-26,2552.0,33.0
116,2022-04-27,2552.0,59.0
117,2022-04-28,2552.0,16.0
118,2022-04-29,2552.0,0.0


In [151]:
# Convert daily to weekly sale

product_orders = product_orders.groupby([pd.Grouper(key='CHECKOUT_DATE', freq='W-MON'), 'PRODUCT_ID']).sum().sort_values(by='CHECKOUT_DATE')
product_orders = product_orders.reset_index().drop(['CHECKOUT_DATE', 'PRODUCT_ID'], axis=1)
product_orders = product_orders[:-1] # Remove the last week because data for the last days are missing

product_orders

,QUANTITY
0,23.0
1,461.0
2,252.0
3,217.0
4,224.0
5,149.0
6,151.0
7,149.0
8,196.0
9,152.0


In [152]:
product_orders['QUANTITY']

0      23.0
1     461.0
2     252.0
3     217.0
4     224.0
5     149.0
6     151.0
7     149.0
8     196.0
9     152.0
10    150.0
11    142.0
12    134.0
13     63.0
14    168.0
15    114.0
16    125.0
Name: QUANTITY, dtype: float64

In [148]:
plot_data = [
    go.Scatter(
        x=product_orders.index,
        y=product_orders['QUANTITY'],
    )
]
plot_layout = go.Layout(
        title='Weekly Sales'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

KeyError: 'QUANTITY'

In [147]:
product_orders = product_orders.T.reset_index().drop(['index'], axis=1)
product_orders.insert(loc=0, column='PRODUCT_ID', value=[id])

product_orders

,PRODUCT_ID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2552,23.0,461.0,252.0,217.0,224.0,149.0,151.0,149.0,196.0,152.0,150.0,142.0,134.0,63.0,168.0,114.0,125.0


In [120]:
pd.concat([product_orders, product_orders], ignore_index=True)

,PRODUCT_ID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2552,23.0,461.0,252.0,217.0,224.0,149.0,151.0,149.0,196.0,152.0,150.0,142.0,134.0,63.0,168.0,114.0,125.0
1,2552,23.0,461.0,252.0,217.0,224.0,149.0,151.0,149.0,196.0,152.0,150.0,142.0,134.0,63.0,168.0,114.0,125.0
